In [4]:
import inflection
from json import loads, load


In [69]:
def to_rust_struct(name, obj):
    desc = obj.get("description", "empty")
    reprs = []
    for k, v in obj["properties"].items():
        rt = to_rust_type(k, v)
        if k not in obj.get("required", []):
            rt = f"Option<{rt}>"
            
        newk = inflection.underscore(k)
        if newk == "type":
            newk = "r#type"
        
        repr = f"        {newk}: {rt},"
        if "ID" in k:
            repr = f"""        #[serde(rename = "{k}")]""" + "\n" + repr
        reprs.append(repr)
    reprs = '\n'.join(reprs)
    s = f"""
    #[derive(Clone, Debug, Deserialize, Serialize)]
    #[serde(rename_all = "camelCase")]
    /// {desc}
    pub struct {name} {{

{reprs}
    }}
    """
    return s

def to_rust_type(keyname, typedesc):
    if "type" in typedesc:
        ty = typedesc["type"]
        if ty == "string":
            if keyname in ["side", "pegPriceType", "ordType", "timeInForce", "execInst", "contingencyType", "binSize"]:
                return inflection.camelize(keyname)
            
            if "format" not in typedesc:
                return "String"
            elif typedesc["format"] == "date-time":
                return "DateTime<Utc>"
            elif typedesc["format"] == "guid":
                return "Uuid"
            else:
                raise RuntimeError(f"Unimplemented for {ty}: {typedesc['format']}")
        elif ty == "number":
            if typedesc["format"] == "int64":
                return "i64"
            if typedesc["format"] == "int32":
                return "i32"
            elif typedesc["format"] == "double":
                return "f64"
            else:
                raise RuntimeError(f"Unimplemented for {ty}: {typedesc['format']}")
        elif ty == "boolean":
            return "bool"
        elif ty == "object":
            return "Value"
        elif ty == "array":
            subtype = to_rust_type(None, typedesc["items"])
            return f"Vec<{subtype}>"
        else:
            raise RuntimeError(f"Unimplemented for {ty}")
    elif "$ref" in typedesc:
        if typedesc["$ref"].startswith("#/definitions/"):
            ty = typedesc["$ref"].replace("#/definitions/", "")
            if ty == "x-any":
                ty = "XAny"
            return ty

In [70]:
for k, v in swagger["definitions"].items():
    try:
        if k == "x-any":
            k = "XAny"
        print(to_rust_struct(k, v))
    except Exception as e:
        print(e)
        print(k, v)



    #[derive(Clone, Debug, Deserialize, Serialize)]
    #[serde(rename_all = "camelCase")]
    /// Public Announcements
    pub struct Announcement {

        id: i32,
        link: Option<String>,
        title: Option<String>,
        content: Option<String>,
        date: Option<DateTime<Utc>>,
    }
    

    #[derive(Clone, Debug, Deserialize, Serialize)]
    #[serde(rename_all = "camelCase")]
    /// empty
    pub struct Error {

        error: Value,
    }
    

    #[derive(Clone, Debug, Deserialize, Serialize)]
    #[serde(rename_all = "camelCase")]
    /// empty
    pub struct XAny {


    }
    

    #[derive(Clone, Debug, Deserialize, Serialize)]
    #[serde(rename_all = "camelCase")]
    /// Persistent API Keys for Developers
    pub struct APIKey {

        id: String,
        secret: String,
        name: String,
        nonce: i64,
        cidr: Option<String>,
        permissions: Option<Vec<XAny>>,
        enabled: Option<bool>,
        user_id: i32,
        created:

In [68]:
with open("../swagger.json") as f:
    swagger = load(f)